In [1]:
# Imports
import os
import sys
import random
import time
import ipywidgets as widgets
# ML Libraries
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.functional as F
# Protein Analysis Libaries
import MDAnalysis as mda
import nglview as nv
# Custom Libraries
from mol_processors.Protein import Prot
from utils import write_array, tensors_to_batch_flat, batch_flat_to_tensors, tensors_to_flat, flat_to_tensors
%pylab inline
%load_ext autoreload 
%autoreload 2
np.set_printoptions(suppress=True)

Populating the interactive namespace from numpy and matplotlib


/home/conradli/miniconda3/envs/hbond/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


# Widgets

In [2]:
# Select all torsions
torsion_select = widgets.Dropdown(
    options=["all", "backbone", "sidechains"],
    value="all",
    description="Selection:",
    disabled=False,
)

# User-Specific Variables

In [3]:
pdb_path = "/home/conradli/SAC-for-H-Bond-Learning/data/pdb/1bdd.pdb"

# Test Loading Proteins

Also run this cell to reset the protein torsion angles

In [62]:
protein = Prot(pdb_path)
print("Total Residues:", protein.total_res)
print("Number of Expected Torsion Angles:", protein.total_res * 7)
print("Torsion Angles Shape:", protein.torsion_angles.shape)
print("Torsion Mask Shape:", protein.torsion_mask.shape)
print("Number of Torsion Active Torsions:", np.sum(protein.torsion_mask == 1))
print("Atom Coords Shape:", protein.coords.shape)
print("Atom Charges Shape:", protein.charges.shape)
print("Atom Radii Shape:", protein.radii.shape)

Total Residues: 60
Number of Expected Torsion Angles: 420
Torsion Angles Shape: (420,)
Torsion Mask Shape: (420,)
Number of Torsion Active Torsions: 242
Atom Coords Shape: (941, 3)
Atom Charges Shape: (941,)
Atom Radii Shape: (941,)


# Test Moving Torsion Angles

In [63]:
display(torsion_select)
indices = protein.select_torsions(selection=torsion_select.value)

Dropdown(description='Selection:', index=2, options=('all', 'backbone', 'sidechains'), value='sidechains')

In [73]:
new_angles = protein.get_torsion_angles()[indices] + 90
protein.set_torsion_angles(indices, new_angles)
protein.update_cartcoords_from_intcoords()
view = nv.show_biopython(protein.chain)
view.add_licorice()
# view.remove_cartoon()
view

NGLWidget()